# Code Sync Utililies
The AWS Console provides nice Web UI for Amazon Web Services, some of which even include script editors.  Web UIs are great, especially for exploration and prototyping. But it is very hard to remember which forms were filled and which buttons were clicked when trying to reproduce swell functionality across environments.  And it quickly becomes difficult to recall which is the 'latest and greatest' script.  An automated, repeatable process is called for.  Fortunately, everything (and more) that can be done on AWS Console can also be done in code.  This notebook contains utilities to 'push' and 'pull' local code 'to' and 'from' [AWS Serverless Services](https://aws.amazon.com/serverless/), including an S3 Code Bucket.  Local code can of course be managed as a Git repository to enable Software Configuration Management (SCM) with tools like Bitbucket, Bamboo, etc.

[AWS CloudFormation](https://docs.aws.amazon.com/cloudformation/) enables us to create and provision AWS service deployments predictably and repeatedly.  CloudFormation utilizes a *template* file to manage a collection of AWS Services as a single unit (i.e., a *stack*.)  [CloudFormation Templates](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/template-guide.html) for some resource types can refer to an S3 object to obtain service source code:
- *Lambda Functions* -- a 'lambda_function.py' python script, and possibly some related files, in a 'deployment package' ZIP file.
- *Step Function State Machines* -- a 'state_machine.json' file that defines the workflow.
- *Glue ETL Jobs* -- a python script file that defines the job.

Utility functions in this notebook utilize a 'stack' global as a prefix for each resource name in the project's [CloudFormation template](ci-cd/cfn_template.yaml).  'stack' can be made to correspond with with a Git feature or branch ID to enable isolated parallel development and testing with a dedicated set of resources.

### CI/CD Considerations
Continuous Integration and Continuous Delivery (CI/CD) implements and enforces automation in building, testing and deployment of applications.  Simplicity of serverless computing makes it conducive to small team projects, which favor less formality and complexity in SCM and CI/CD.  CI/CD for serverless projects on AWS should utilize CloudFormation, or its extension the [AWS Serverless Application Model (SAM)](https://aws.amazon.com/serverless/sam/).

Here is a list of steps to Clone the project in AWS by defining a new and unique CloudFormation 'stack' for a feature or branch.
- Set up an S3 Code Bucket (or folder) for the feature or branch, and push S3 objects expected by the CloudFormation template.  
- Create the CloudFormation feature stack
- develop serverless code mods
- Sync mod'd code to local development (if edited on AWS) 
- Pushed sync'd code to stack's S3 Code Bucket
- Modify other CloudFormation stack properties , if needed 
- Update the CloudFormation 'feature' 
- Check-in and Merge tested code, including Cfn Stack, into Git Master
- Sync stack's S3 Code Bucket into Master Code Bucket (Bamboo)
- Update the CloudFormation 'master' stack (Bamboo)


### Using Jupyter Notebook Features
Each of the functions is defined in a separate cell.   Each function definition is followed by a print statement that provides a 'Usage' example.   Comment the print statement line to make the example code 'runnable' interactively.  Or copy/paste the usage examples into new cells to call the functions.

In [97]:
# Initialize Persistent %store & Notebook Globals
import json
import boto3
import os

# 'persisted' variables accessible to ALL local notebooks ...
S_stack = "daab-lab-smpl-main"
S_rootdir = "/home/ec2-user/SageMaker/daab-simple"
S_partition = 'aws'
S_region = 'us-east-2'
S_account_id = '{aws_acct}'

# prior ...
#S_stack = "daab-lab-fp5a"
#S_region = 'us-east-1'

S_arn_template = f"arn:{S_partition}:$Service:{S_region}:{S_account_id}:$Resource"
#S_code_bucket = f"{S_stack}-code"
S_code_bucket = "daab-lab-code-us-east-2"

S_qualifier  = "FDMD" # for DEV, or FDMQ for Test, FDMP for Prod
S_sys_abbrev = "FSDATA"
S_file_prefix = f"{S_qualifier}.{S_sys_abbrev}"
S_glue_dbname = f"{S_stack}-{S_sys_abbrev.lower()}"

# ... ref https://ipython.readthedocs.io/en/stable/config/extensions/storemagic.html
%store -z
%store S_stack S_rootdir S_partition S_region S_account_id S_arn_template S_code_bucket S_qualifier S_sys_abbrev S_file_prefix S_glue_dbname
%store

# Globals
#s3_code_bucket = f"{S_stack}-code"
os.environ['AWS_DEFAULT_REGION'] = S_region

def pp(x):
    x_s = json.dumps(x, indent=2, default=str)
    print(x_s)
    return x_s


Stored 'S_stack' (str)
Stored 'S_rootdir' (str)
Stored 'S_partition' (str)
Stored 'S_region' (str)
Stored 'S_account_id' (str)
Stored 'S_arn_template' (str)
Stored 'S_code_bucket' (str)
Stored 'S_qualifier' (str)
Stored 'S_sys_abbrev' (str)
Stored 'S_file_prefix' (str)
Stored 'S_glue_dbname' (str)
Stored variables and their in-db values:
S_account_id               -> '{aws_acct}'
S_arn_template             -> 'arn:aws:$Service:us-east-2:{aws_acct}:$Resource
S_code_bucket              -> 'daab-lab-code-us-east-2'
S_file_prefix              -> 'FDMD.FSDATA'
S_glue_dbname              -> 'daab-lab-smpl-main-fsdata'
S_partition                -> 'aws'
S_qualifier                -> 'FDMD'
S_region                   -> 'us-east-2'
S_rootdir                  -> '/home/ec2-user/SageMaker/daab-simple'
S_stack                    -> 'daab-lab-smpl-main'
S_sys_abbrev               -> 'FSDATA'


In [98]:
def substitute_parms( template_string, parms_dict ):
    ''' Replace Dictionary Keys with Dictionary Values '''
    for key,val in parms_dict.items():
        template_string = template_string.replace(key,val)

    return template_string

pull_parms = {
    f":{S_partition}:" : ":$Partition:",
    f":{S_region}:" : ":$Region:",
    f":{S_account_id}:" : ":$AccountId:",
    S_stack : "$Stack",
    S_qualifier : "$Qualifier",
    S_sys_abbrev : "$SysAbbrev",
    S_glue_dbname : "$GlueDbName"
}
push_parms = {
    ":$Partition:" : f":{S_partition}:",
    ":$Region:" : f":{S_region}:",
    ":$AccountId:" : f":{S_account_id}:",
    "$Stack" : S_stack,
    "$Qualifier" : S_qualifier,
    "$SysAbbrev" : S_sys_abbrev,
    "$GlueDbName" : S_glue_dbname
}
#print ( '''
# Usage:
print( f"Template: {S_arn_template}" )
pulled_arn = substitute_parms ( S_arn_template , pull_parms )
print( "  Pulled: " + pulled_arn  )
print( "  Pushed: " + substitute_parms ( pulled_arn , push_parms ) )

# Sample Output:
#Template: arn:aws:$Service:us-east-2:{aws_acct}:$Resource
#  Pulled: arn:$Partition:$Service:$Region:$AccountId:$Resource
#  Pushed: arn:aws:$Service:us-east-2:{aws_acct}:$Resource
# ''' )

Template: arn:aws:$Service:us-east-2:{aws_acct}:$Resource
  Pulled: arn:$Partition:$Service:$Region:$AccountId:$Resource
  Pushed: arn:aws:$Service:us-east-2:{aws_acct}:$Resource


In [18]:
def pull_stepfunction( local_path, from_arn):
    ''' pull Step Function ARN definition to local'''
    sfn_client = boto3.client('stepfunctions')

    if not os.path.exists(local_path):
        os.makedirs( local_path )

    response = sfn_client.describe_state_machine(
        stateMachineArn=from_arn
    )
    state_machine_name = response['name']
    state_machine_role_arn = response['roleArn']
    state_machine_json = response['definition']

    state_machine_json = substitute_parms( state_machine_json, pull_parms )
    
    with open(f'{local_path}/state_machine.json', 'w') as f:
        f.write(state_machine_json)
        
    #x = pp(response)
    print(f"Downloaded '{from_arn}' to '{local_path}' ...")
    print(os.listdir(local_path))

    return response

#print ( ''' 
# Usage: 
arn_template = S_arn_template.replace("$Service", "states")
functions = [ 
    "Extract_Zip_to_Parquet"
]0

for function in functions:
    local_path = f"{S_rootdir}/stepfunctions/{function.split('-')[-1]}"
    function_arn = arn_template.replace("$Resource", f"stateMachine:{S_stack}-{function}")

    response = pull_stepfunction( local_path, function_arn)

# Sample Response:
# Downloaded 'arn:aws:states:us-east-1:{aws_acct}:stateMachine:daab-lab-fp5a-Extract_Zip_to_Parquet' to 'C:/Users/richa/Code/AwsGlueWorkbook/stepfunctions/Extract_Zip_to_Parquet' ...
# ['state_machine.json']
#''' )


Downloaded 'arn:aws:states:us-east-1:{aws_acct}:stateMachine:daab-lab-fp5a-Extract_Zip_to_Parquet' to '/home/ec2-user/SageMaker/daab-simple/stepfunctions/Extract_Zip_to_Parquet' ...
['state_machine.json', '.ipynb_checkpoints']


In [8]:
def pull_lambda( from_arn, local_path ):
    ''' pull Lambda Function ARN code to local'''
    import requests
    import zipfile
    import io
    import json
    import os

    lambda_client = boto3.client('lambda')

    if not os.path.exists(local_path):
        os.makedirs( local_path )

    response = lambda_client.get_function(
        FunctionName=from_arn
    )
    # download & extract Lambda code zipfile ...
    # ref https://stackoverflow.com/questions/9419162/download-returned-zip-file-from-url
    zipbytes = requests.get(response['Code']['Location'], stream=True)
    z = zipfile.ZipFile(io.BytesIO(zipbytes.content))
    z.extractall(local_path)

    # ... and write a local copy of the function's configuration
    config_path = f"{local_path}/lambda_config.json"
    with open (config_path, 'w') as f:
        f.write(json.dumps(response['Configuration'], indent=2))

    print(f"Downloaded '{from_arn}' to '{local_path}' ...")
    print(os.listdir(local_path))

    return response

#print ( '''
# Usage: 
arn_template = S_arn_template.replace("$Service", "lambda")
functions = [ 
    "S3_Unzip",
    "Process_Initiator"
]
for function in functions:
    local_path = f"{S_rootdir}/lambda/{function.split('-')[-1]}"
    function_arn = arn_template.replace("$Resource", f"function:{S_stack}-{function}")

    response = pull_lambda( function_arn, local_path )

# Sample Response
# Downloaded 'arn:aws:lambda:us-east-1:{aws_acct}:function:daab-lab-fp5a-S3_Unzip' to '/home/ec2-user/SageMaker/daab-simple/lambda/S3_Unzip' ...
# ['lambda_config.json', 'lambda_function.py']
# Downloaded 'arn:aws:lambda:us-east-1:{aws_acct}:function:daab-lab-fp5a-Process_Initiator' to '/home/ec2-user/SageMaker/daab-simple/lambda/Process_Initiator' ...
# ['lambda_config.json', 'lambda_function.py']
#''' )


Downloaded 'arn:aws:lambda:us-east-1:{aws_acct}:function:daab-lab-fp5a-S3_Unzip' to '/home/ec2-user/SageMaker/daab-simple/lambda/S3_Unzip' ...
['lambda_config.json', 'lambda_function.py']
Downloaded 'arn:aws:lambda:us-east-1:{aws_acct}:function:daab-lab-fp5a-Process_Initiator' to '/home/ec2-user/SageMaker/daab-simple/lambda/Process_Initiator' ...
['lambda_config.json', 'lambda_function.py', '.ipynb_checkpoints']


In [24]:
def push_stepfunction( local_path, to_arn):
    ''' push local Step Function definition to AWS ARN'''
    sfn_client = boto3.client('stepfunctions')

    with open( f"{local_path}/state_machine.json", "r") as f:
        state_machine_json = f.read()

    push_parms = {
        ":$Partition:" : f":{S_partition}:",
        ":$Region:" : f":{S_region}:",
        ":$AccountId:" : f":{S_account_id}:",
        "$Stack" : f"{S_stack}"
    }
    state_machine_json = substitute_parms( state_machine_json, push_parms )
        
    try:
        response = sfn_client.update_state_machine(
            stateMachineArn = to_arn,
            definition = state_machine_json
        )
    except sfn_client.exceptions.StateMachineDoesNotExist:
        sfn_name = f"{S_stack}-{function}"
        response = sfn_client.create_state_machine(
            name = sfn_name,
            definition = state_machine_json,
            roleArn = f"arn:aws:iam::{aws_acct}:role/{S_stack}-StatesExecutionRole"
        )

    print(f"Uploaded '{local_path}' to '{to_arn}'.")
    #print(os.listdir(local_path))
    
    return response

#print ( '''
# Usage: 
arn_template = S_arn_template.replace("$Service", "states")
functions = [ 
    "Extract_Zip_to_Parquet"
]
for function in functions:
    local_path = f"{S_rootdir}/stepfunctions/{function.split('-')[-1]}"
    function_arn = arn_template.replace("$Resource", f"stateMachine:{S_stack}-{function}")

    response = push_stepfunction( local_path, function_arn)

# Sample Response:
# Uploaded 'C:/Users/richa/Code/AwsGlueWorkbook/stepfunctions/Extract_Zip_to_Parquet' to 'arn:aws:states:us-east-1:{aws_acct}:stateMachine:daab-lab-fp5a-Extract_Zip_to_Parquet'.
#''' )


Uploaded '/home/ec2-user/SageMaker/daab-simple/stepfunctions/Extract_Zip_to_Parquet' to 'arn:aws:states:us-east-2:{aws_acct}:stateMachine:daab-lab-smpl-main-Extract_Zip_to_Parquet'.


In [103]:
def push_lambda( local_path, to_arn, s3_code_folder=''):
    ''' push local Lambda Function definition to AWS ARN'''
    import shutil

    if s3_code_folder == '':
        s3_code_folder = f"{S_stack}/lambda"

    zipfile_name = f"{local_path.split('/')[-1]}.zip"
    
    lambda_client = boto3.client('lambda')
    s3_client = boto3.client('s3')
    s3_key = f"{s3_code_folder}/{zipfile_name}"
    
    # Zip local code and upload it ...
    shutil.make_archive(f"{local_path}", "zip", local_path)
    s3_client.upload_file( f"{local_path}.zip", S_code_bucket, s3_key )
    print(f"Uploaded '{local_path}' to 's3://{S_code_bucket}/{s3_key}'")

    # ... update Lambda function from uploaded Zip 
    response = lambda_client.update_function_code(
        FunctionName = to_arn,
        S3Bucket = S_code_bucket,
        S3Key = s3_key
    )
    '''
    # ToDo: also update other parms from 'lambda_config.json'
    response = lambda_client.update_function_configuration(  
        FunctionName = to_arn,
        ...
    )
    '''
    print(f"Updated Code in '{to_arn}'")    

    return response

#print ( '''
# Usage: 
arn_template = S_arn_template.replace("$Service", "lambda")

functions = [ 
    "S3_Unzip",
    "Process_Initiator"
]
for function in functions:
    local_path = f"{S_rootdir}/lambda/{function}"
    function_arn = arn_template.replace("$Resource", f"function:{S_stack}-{function}" )
    #print ( local_path, function_arn)

    response = push_lambda( local_path, function_arn)

# Sample Output:
# Uploaded 'C:/Users/richa/Code/AwsGlueWorkbook/lambda/S3_Unzip' to 's3://daab-lab-fp5a-code/daab-lab-fp5a/lambda/S3_Unzip.zip'
# Updated Code in 'arn:aws:lambda:us-east-1:{aws_acct}:function:daab-lab-fp5a-S3_Unzip'    
#''' )


Uploaded '/home/ec2-user/SageMaker/daab-simple/lambda/S3_Unzip' to 's3://daab-lab-code-us-east-2/daab-lab-smpl-main/lambda/S3_Unzip.zip'
Updated Code in 'arn:aws:lambda:us-east-2:{aws_acct}:function:daab-lab-smpl-main-S3_Unzip'
Uploaded '/home/ec2-user/SageMaker/daab-simple/lambda/Process_Initiator' to 's3://daab-lab-code-us-east-2/daab-lab-smpl-main/lambda/Process_Initiator.zip'
Updated Code in 'arn:aws:lambda:us-east-2:{aws_acct}:function:daab-lab-smpl-main-Process_Initiator'


In [ ]:
def pull_glue_job(from_arn, local_path ):
    ''' pull Glue Job script to local '''
    glue_client = boto3.client('glue')
    s3_client = boto3.client('s3')
    
    response = glue_client.get_job( JobName = from_arn )
        
    script_loc = response['Job']['Command']['ScriptLocation']

    fname = script_loc[script_loc.find('//')+2:]
    s3_bucket = fname[:fname.find('/')]
    s3_key = fname[fname.find('/')+1:]

    s3_client.download_file( s3_bucket, s3_key, local_path )
    
    print(f"Downloaded '{script_loc}' to '{local_path}'")
    #print(os.listdir(local_path))

    return response

#print ('''
# Usage:
glue_job = 'daab-lab-fp5a-Convert_CSV_To_Parquet'

response = pull_glue_job( glue_job , f"{S_rootdir}/glue/{glue_job.split('-')[-1]}.py")
           
print(json.dumps(response['Job'], indent=2, default=str))
  
# Sample Output
# Downloaded 's3://daab-lab-code-us-east-1/daab-lab-fp5a/glue/Convert_CSV_To_Parquet.py' to '/home/ec2-user/SageMaker/daab-simple/glue/Convert_CSV_To_Parquet.py'
#''' )

In [54]:
def pull_ebrule( eb_rulename ) :
    eb_client = boto3.client('events')

    outfolder = f"{S_rootdir}/eventbridge/{S_file_prefix}"
    if not os.path.exists(outfolder):
        os.makedirs(outfolder)

    response = eb_client.describe_rule( Name = eb_rulename )
    del response['ResponseMetadata']
    rule_json = substitute_parms( json.dumps(response, indent=2) , pull_parms )

    rule_filename = f"{outfolder}/rule.json"
    with open ( rule_filename , "w" ) as f:
        f.write (rule_json)
        
    print( f"Pulled Rule '{eb_rulename}' to '{rule_filename}'")

    response = eb_client.list_targets_by_rule(
        Rule=eb_rulename,
    )
    targets_json = substitute_parms( json.dumps(response['Targets'], indent=2), pull_parms )

    targets_filename = f"{outfolder}/targets.json"
    with open ( targets_filename , "w" ) as f:
        f.write (targets_json)

    print( f"Pulled '{eb_rulename}' Targets to '{targets_filename}'")

# Usage:
pull_ebrule( 'daab-lab-fp5a-FDMD.FSDATA' ) 

    


Pulled Rule 'daab-lab-fp5a-FDMD.FSDATA' to '/home/ec2-user/SageMaker/daab-simple/eventbridge/FDMD.FSDATA/rule.json'
Pulled 'daab-lab-fp5a-FDMD.FSDATA' Targets to '/home/ec2-user/SageMaker/daab-simple/eventbridge/FDMD.FSDATA/targets.json'


In [55]:
def list_cfn_stack_resources (cfn_stackname ):
    import boto3

    cfn_client = boto3.client('cloudformation')

    response = cfn_client.describe_stack_resources(
        StackName=cfn_stackname,
        #LogicalResourceId='string',
        #PhysicalResourceId='string'
    )

    print( f"Resources in Stack {cfn_stackname}:")
    for resource in response['StackResources']:
        print( f"{resource['LogicalResourceId']:<20}\t{resource['ResourceType']:<20}\t{resource['PhysicalResourceId']}".expandtabs(16))

    return response

# Usage:
response = list_cfn_stack_resources( S_stack )


Resources in Stack daab-lab-fp5a:
EventRuleFDMDxFISCALDATA        AWS::Events::Rule               daab-lab-fp5a-FDMD.FISCALDATA
GlueFiscalDataDB                AWS::Glue::Database             daab-lab-fp5a-top-fiscaldata
GlueJobConvertCsvToParquet      AWS::Glue::Job                  daab-lab-fp5a-Convert_CSV_To_Parquet
GlueJobRole                     AWS::IAM::Role                  daab-lab-fp5a-Glue_Job_Service_Role
LambdaProcessInitiatorFunction  AWS::Lambda::Function           daab-lab-fp5a-Process_Initiator
LambdaProcessInitiatorRole      AWS::IAM::Role                  daab-lab-fp5a-Process_Initiator_Role
LambdaS3UnzipFunction           AWS::Lambda::Function           daab-lab-fp5a-S3_Unzip
LambdaS3UnzipRole               AWS::IAM::Role                  daab-lab-fp5a-S3_Unzip_Role
S3DataLakeBucket                AWS::S3::Bucket                 daab-lab-fp5a-datalake
S3LandingPadBucket              AWS::S3::Bucket                 daab-lab-fp5a-landing-pad
StateMachineExtractZipToP

In [105]:
def put_cfn_stack ( cfn_stackname , cfn_template_url, cfn_parms = []  ):
    ''' CRdate or UPdate CloudFormation Stack '''
    # ToDo - use SAM deploy instead of Cfn Create/Update (e.g., fp4)?
    import boto3

    cfn_client = boto3.client('cloudformation')

    try:
        response = cfn_client.create_stack(
            StackName= cfn_stackname,
            TemplateURL= cfn_template_url,
            Parameters= cfn_parms,
            Capabilities=[ 'CAPABILITY_NAMED_IAM' ]
        )
        status = 'Created'
    except cfn_client.exceptions.AlreadyExistsException:
        response = cfn_client.update_stack(
            StackName= cfn_stackname,
            TemplateURL= cfn_template_url,
            Parameters= cfn_parms,
            Capabilities=[ 'CAPABILITY_NAMED_IAM' ]
        )
        status = 'Updated'

    print( f"{status} Stack '{cfn_stackname}' using Template '{cfn_template_url}'" )
    return response

# Usage:
#S_stack = 'daab-lab-smpl-main'

# Copy CloudFormation Template & Script Files it References to Code Bucket
s3_client = boto3.client('s3')
files = [
    'ci-cd/cfn_template.yaml',
    'glue/Convert_CSV_To_Parquet.py',
    'lambda/Process_Initiator.zip',
    'lambda/S3_Unzip.zip',
    'stepfunctions/Extract_Zip_to_Parquet/state_machine.json'
]
for file in files:
    s3_key = f"{S_stack}/{file}"
    filetype = file.split('.')[-1]
    if filetype in ('yaml','json'):
        with open( f"{S_rootdir}/{file}", "r" ) as f:
            template = f.read()
        template = substitute_parms( template, push_parms )
        print( f"Pushed parms to '{S_rootdir}/{file}'" )
        
        s3_client.put_object( Bucket=S_code_bucket, Key=s3_key, Body=template )
    else:
        s3_client.upload_file( f"{S_rootdir}/{file}", S_code_bucket, s3_key )

    print( f"Uploaded '{S_rootdir}/{file}' to 's3://{S_code_bucket}/{s3_key}" )
    
s3_url = f'https://{S_code_bucket}.s3.amazonaws.com/{S_stack}/ci-cd/cfn_template.yaml'

response = put_cfn_stack( S_stack, s3_url )

Pushed parms to '/home/ec2-user/SageMaker/daab-simple/ci-cd/cfn_template.yaml'
Uploaded '/home/ec2-user/SageMaker/daab-simple/ci-cd/cfn_template.yaml' to 's3://daab-lab-code-us-east-2/daab-lab-smpl-main/ci-cd/cfn_template.yaml
Uploaded '/home/ec2-user/SageMaker/daab-simple/glue/Convert_CSV_To_Parquet.py' to 's3://daab-lab-code-us-east-2/daab-lab-smpl-main/glue/Convert_CSV_To_Parquet.py
Uploaded '/home/ec2-user/SageMaker/daab-simple/lambda/Process_Initiator.zip' to 's3://daab-lab-code-us-east-2/daab-lab-smpl-main/lambda/Process_Initiator.zip
Uploaded '/home/ec2-user/SageMaker/daab-simple/lambda/S3_Unzip.zip' to 's3://daab-lab-code-us-east-2/daab-lab-smpl-main/lambda/S3_Unzip.zip
Pushed parms to '/home/ec2-user/SageMaker/daab-simple/stepfunctions/Extract_Zip_to_Parquet/state_machine.json'
Uploaded '/home/ec2-user/SageMaker/daab-simple/stepfunctions/Extract_Zip_to_Parquet/state_machine.json' to 's3://daab-lab-code-us-east-2/daab-lab-smpl-main/stepfunctions/Extract_Zip_to_Parquet/state_mac

ClientError: An error occurred (ValidationError) when calling the UpdateStack operation: No updates are to be performed.